In [37]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import random

In [38]:
url = 'https://www.amazon.com/s?k=jacket+for+men&ref=nb_sb_noss'

In [39]:
r = requests.get('http://localhost:8000/render.html', {'url': url, 'wait': 2})
print(r)

<Response [200]>


In [40]:
soup = BeautifulSoup(r.text, 'html.parser')
string = 'Sorry, we just need to make sure you\'re not a robot. For best results, please make sure your browser is accepting cookies'
string in soup.find('body').text

False

In [5]:
product_links = pd.DataFrame(columns=['product', 'link'])
i = 0

url = 'https://www.amazon.com/s?k=jacket+for+men&ref=nb_sb_noss'

for page in range(1, 1000):

    print("scraping page - ", page)
    
    r = requests.get('http://localhost:8000/render.html', {'url': url, 'wait': 3})
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # stop if next page button is disabled
    if soup.find('li', {'class': 'a-disabled a-last'}):
        break
    else:
        # go to next page
        # find element of next page
        next_links = soup.find_all('li', {'class':'a-last'})
        for link in next_links:
            url = 'https://amazon.com' + str(link.find('a')['href'])
    
    products = soup.find_all('div', {'data-component-type': 's-search-result'})

    for product in products:
        item = product.find('a', {'class':'a-link-normal s-underline-text s-underline-link-text a-text-normal'})
        if item == None:
            item = product.find('a', {'class':'a-link-normal a-text-normal'})
        title = item.text.strip()
        link = item['href']
        product_links.loc[i] = [title, link]
        i = i+1
    if not page == 1:
        soup = soup.find_all('li', {'class':'a-normal'})

print('done')
product_links.to_csv('scraped_product_links.csv', index=False)

scraping page -  1
scraping page -  2
scraping page -  3
scraping page -  4
scraping page -  5
scraping page -  6
scraping page -  7
done


In [9]:
products = pd.read_csv('products_complete.csv')

#products['features'] = ''
#products['description'] = ''
#products['review_pages'] = ''

for i in range(110, len(products)):
    
    print("", round((100 * i) / len(products)), "% product pages scraped")
    
    url_base = products['link'].iloc[i]
    url = 'https://amazon.com' + url_base

    r = requests.get('http://localhost:8000/render.html', {'url': url, 'wait': random.randint(2,20)})
    soup = BeautifulSoup(r.text, 'html.parser')
    
    try:
        if str(r) == "<Response [503]>" or string in soup.find('body').text:
            print('#####amazon is blocking you######')
            print('no of pages scraped - ',i )
            break
    except:
        print('#####amazon is blocking you######')
        print('no of pages scraped - ',i )
        break

    product_features = ''
    product_description = ''
    review_page_link = ''
    
    review_item = soup.find({'a'}, {'data-hook':"see-all-reviews-link-foot"})
    
    if review_item is None:          
        products['features'][i] = product_features
        products['description'][i] = product_description
        products['review_pages'][i] = review_page_link
        print('Not enough reviews - adding empty entries')
        continue
        
    review_page_link = review_item['href']

    feature_list_conatiner = soup.find({'div'}, {'id':"feature-bullets"})
    feature_list = feature_list_conatiner.find_all({'span'}, {'class':"a-list-item"})
    for feature in feature_list:
        product_features = product_features + " " + feature.text.strip()

    product_description_container = soup.find('div', 
                                              {'cel_widget_id':
                                               "dpx-aplus-3p-product-description_csm_instrumentation_wrapper"})    
    
    if product_description_container == None:
        product_description_container = soup.find('div', {'id':'productDescription'})

    if product_description_container == None:
        products['features'][i] = product_features
        products['description'][i] = product_description
        products['review_pages'][i] = review_page_link
        continue
    
    lists = product_description_container.find_all({'span'}, {'class':"a-list-item"})
    
    for item in lists:
        product_description = product_description + " " + item.text.strip()

    paras = product_description_container.find_all('p')
    for para in paras:
        if '<img alt=' in para.text:
            continue
        product_description = product_description + " " + para.text.strip()
    
    products['features'][i] = product_features
    products['description'][i] = product_description
    products['review_pages'][i] = review_page_link

 30 % product pages scraped
 30 % product pages scraped
 31 % product pages scraped
#####amazon is blocking you######
no of pages scraped -  110


In [10]:
soup

{"error": 504, "type": "GlobalTimeoutError", "description": "Timeout exceeded rendering page", "info": {"remaining": -0.013931035995483398, "timeout": 30}}

In [11]:
i

110

In [12]:
products.to_csv('products_complete.csv', index = False)

In [52]:
def clean_title(item):
    if item == None:
        return ""
    else:
        return item.text.strip().replace('\n', '')
    
def clean_rating(item):
    if item == None:
        return 0.0
    else:
        return float(item.text[:3])

def clean_body(item):
    if item == None:
        print("empty body")
        return ""
    else:
        return item.text.strip()
df = pd.DataFrame(columns = ['product', 'review_title', 'review_rating', 'review_text'])
#df = pd.read_csv('scraped_data2.csv')


#first 450 should be scraped again
products = pd.read_csv('products_complete.csv')
i = 0#len(df)

for prd_idx in range(len(products)):
    print('scraping reviews of product - ', prd_idx + 1)
    url_base = products['review_pages'][prd_idx]
    
    if url_base == "":
        continue
    if type(url_base) == type(2.0):
        continue
    
    for page in range(20):
        print('reviews scrapped - ', i)
        url = 'https://amazon.com' + url_base
        r = requests.get('http://localhost:8000/render.html', {'url': url, 'wait': random.randint(2,10)})
        
        soup = BeautifulSoup(r.text, 'html.parser')
        
            # stop if next page button is disabled
        if soup.find('li', {'class': 'a-disabled a-last'}):
            break
        else:
            # go to next page
            # find element of next page
            next_links = soup.find_all('li', {'class':'a-last'})
            for link in next_links:
                url_base = str(link.find('a')['href'])
        
        try:
            if str(r) == "<Response [503]>" or string in soup.find('body').text:
                print('#####amazon is blocking you######')
                print('no of pages scraped - ',i )
                break
        except:
            print('#####amazon is blocking you######')
            print('no of pages scraped - ',i )
            break

        #check if we have reached last page
        if soup.find('li', {'class': 'a-disabled a-last'}):
            break

        reviews = soup.find_all('div', {'data-hook': 'review'})

        for items in reviews:
            product = soup.title.text.replace('Amazon.com: Customer reviews:', '').strip()
            reviews_title= clean_title(items.find('a', {'data-hook': 'review-title'}))
            review_rating= clean_rating(items.find('i', {'data-hook': 'review-star-rating'}))
            reviews_text= clean_body(items.find('span', {'data-hook': 'review-body'}))
            df.loc[i] = [product, reviews_title, review_rating, reviews_text]
            i = i + 1

df = df.drop_duplicates()
df.to_csv('scraped_data3.csv', index=False)

scraping reviews of product -  1
reviews scrapped -  0
reviews scrapped -  10
reviews scrapped -  20
reviews scrapped -  30
reviews scrapped -  40
reviews scrapped -  50
reviews scrapped -  60
reviews scrapped -  70
reviews scrapped -  80
reviews scrapped -  90
reviews scrapped -  100
reviews scrapped -  110
reviews scrapped -  120
reviews scrapped -  130
reviews scrapped -  140
reviews scrapped -  150
reviews scrapped -  160
reviews scrapped -  170
reviews scrapped -  180
reviews scrapped -  190
scraping reviews of product -  2
reviews scrapped -  200
reviews scrapped -  210
reviews scrapped -  220
reviews scrapped -  230
reviews scrapped -  240
reviews scrapped -  250
reviews scrapped -  260
reviews scrapped -  270
reviews scrapped -  280
reviews scrapped -  290
reviews scrapped -  300
reviews scrapped -  310
reviews scrapped -  320
reviews scrapped -  330
reviews scrapped -  340
reviews scrapped -  350
reviews scrapped -  360
reviews scrapped -  370
reviews scrapped -  380
reviews s

reviews scrapped -  2890
reviews scrapped -  2900
reviews scrapped -  2910
reviews scrapped -  2920
reviews scrapped -  2930
reviews scrapped -  2940
reviews scrapped -  2950
reviews scrapped -  2960
reviews scrapped -  2970
reviews scrapped -  2980
reviews scrapped -  2990
reviews scrapped -  3000
reviews scrapped -  3010
reviews scrapped -  3020
reviews scrapped -  3030
reviews scrapped -  3040
reviews scrapped -  3050
reviews scrapped -  3060
reviews scrapped -  3070
scraping reviews of product -  22
reviews scrapped -  3080
reviews scrapped -  3090
scraping reviews of product -  23
reviews scrapped -  3090
reviews scrapped -  3100
reviews scrapped -  3110
reviews scrapped -  3120
reviews scrapped -  3130
reviews scrapped -  3140
reviews scrapped -  3150
reviews scrapped -  3160
reviews scrapped -  3170
reviews scrapped -  3180
reviews scrapped -  3190
reviews scrapped -  3200
reviews scrapped -  3210
reviews scrapped -  3220
reviews scrapped -  3230
reviews scrapped -  3240
reviews

reviews scrapped -  5860
reviews scrapped -  5870
reviews scrapped -  5880
reviews scrapped -  5890
reviews scrapped -  5900
reviews scrapped -  5910
reviews scrapped -  5920
reviews scrapped -  5930
reviews scrapped -  5940
reviews scrapped -  5950
reviews scrapped -  5960
reviews scrapped -  5970
reviews scrapped -  5980
reviews scrapped -  5990
reviews scrapped -  6000
reviews scrapped -  6010
scraping reviews of product -  42
reviews scrapped -  6010
reviews scrapped -  6020
reviews scrapped -  6030
reviews scrapped -  6040
reviews scrapped -  6050
reviews scrapped -  6060
reviews scrapped -  6070
reviews scrapped -  6080
scraping reviews of product -  43
reviews scrapped -  6080
reviews scrapped -  6090
reviews scrapped -  6100
reviews scrapped -  6110
reviews scrapped -  6120
reviews scrapped -  6130
reviews scrapped -  6140
reviews scrapped -  6150
reviews scrapped -  6160
reviews scrapped -  6170
reviews scrapped -  6180
reviews scrapped -  6190
reviews scrapped -  6200
reviews

reviews scrapped -  8458
reviews scrapped -  8468
reviews scrapped -  8478
reviews scrapped -  8488
reviews scrapped -  8498
reviews scrapped -  8508
reviews scrapped -  8518
reviews scrapped -  8528
reviews scrapped -  8538
reviews scrapped -  8548
reviews scrapped -  8558
reviews scrapped -  8568
reviews scrapped -  8578
reviews scrapped -  8588
reviews scrapped -  8598
reviews scrapped -  8608
scraping reviews of product -  68
reviews scrapped -  8618
reviews scrapped -  8628
scraping reviews of product -  69
reviews scrapped -  8628
reviews scrapped -  8638
reviews scrapped -  8648
reviews scrapped -  8658
reviews scrapped -  8668
reviews scrapped -  8678
reviews scrapped -  8688
reviews scrapped -  8698
reviews scrapped -  8708
reviews scrapped -  8718
reviews scrapped -  8728
reviews scrapped -  8738
reviews scrapped -  8748
reviews scrapped -  8758
reviews scrapped -  8768
reviews scrapped -  8778
reviews scrapped -  8788
reviews scrapped -  8798
reviews scrapped -  8808
reviews

reviews scrapped -  10768
reviews scrapped -  10778
reviews scrapped -  10788
reviews scrapped -  10798
reviews scrapped -  10808
scraping reviews of product -  92
scraping reviews of product -  93
reviews scrapped -  10808
reviews scrapped -  10818
reviews scrapped -  10828
reviews scrapped -  10838
reviews scrapped -  10848
reviews scrapped -  10858
reviews scrapped -  10868
reviews scrapped -  10878
reviews scrapped -  10888
reviews scrapped -  10898
reviews scrapped -  10908
reviews scrapped -  10918
reviews scrapped -  10928
reviews scrapped -  10938
reviews scrapped -  10948
reviews scrapped -  10958
reviews scrapped -  10968
reviews scrapped -  10978
reviews scrapped -  10988
reviews scrapped -  10998
scraping reviews of product -  94
reviews scrapped -  11008
reviews scrapped -  11018
reviews scrapped -  11028
reviews scrapped -  11038
reviews scrapped -  11048
reviews scrapped -  11058
reviews scrapped -  11068
reviews scrapped -  11078
reviews scrapped -  11088
reviews scrapp

In [54]:
df.duplicated().sum()

0